In [16]:
from src.MOF_builder._learn_template import extract_type_atoms_fcoords_in_primitive_cell,extract_unit_cell,make_supercell_3x3x3,clust_analysis_points,filter_overlapping_points
import numpy as np
#template_cif_file = "mil101_cleaved.cif"
#eenode, _, _ = extract_cluster_center_from_templatecif(
#    template_cif_file, "Y", 6,cluster_distance_threshhold=4
#) 
def analyze_metal_node(template_cif_file, metal_symbol, cluster_size,npyfile_name):
    cell_info, array_atom, array_target_atoms = (
            extract_type_atoms_fcoords_in_primitive_cell(template_cif_file, metal_symbol)
        )
    unit_cell = extract_unit_cell(cell_info)
    # unit_cell = np.round(unit_cell,3)
    metal333 = make_supercell_3x3x3(array_target_atoms)
    metal333 = np.vstack(metal333)
    # cluster analysis in cartesian coordinates
    array_metal_ccords = np.dot(unit_cell, metal333.T).T
#for each atom, find the closest 2 atoms, these 3 atoms are the cluster, then find the center of the cluster, one atom can only be in one cluster

    array = array_metal_ccords
    centers = []
    matched_atoms = []
    for i in range(len(array)):
        if i in matched_atoms:
            continue
        atom = array[i]
        dist = np.linalg.norm(array-atom,axis=1)
        idx = np.argsort(dist)
        cluster = array[idx[:cluster_size]]
        matched_atoms.extend(idx[:cluster_size])
        center = np.mean(cluster,axis=0)
        centers.append(center)
        print(len(matched_atoms)/cluster_size)
    #remove overlapping points
    cluster_centers_ccoords = np.vstack(centers)
    # cluster_centers should return fractional coordinates
    cluster_centers_fcoords = np.dot(
        np.linalg.inv(unit_cell), cluster_centers_ccoords.T
    ).T
    # filter cluster centers which is inside the unit cell, boundary condition is [-0.01,1.01]

    cluster_centers_fcoords = np.mod(cluster_centers_fcoords, 1)
    # cluster_centers_fcoords = [c for c in cluster_centers_fcoords if all([i>=-0.01 and i<=1.01 for i in c])]
    #use np.isclose to filter overlapping points
    cluster_centers_fcoords_neat = cluster_centers_fcoords#[0:280] #if you have a guess range
    for i in range(cluster_centers_fcoords.shape[0]): #if you have a guess range then change to 280:cluster_centers_fcoords.shape[0]
        if not any(np.isclose(cluster_centers_fcoords[i],cluster_centers_fcoords_neat,atol=0.07).all(axis=1)):
            cluster_centers_fcoords_neat = np.vstack([cluster_centers_fcoords_neat,cluster_centers_fcoords[i]])

    print(cluster_centers_fcoords_neat.shape)
    print(cluster_centers_fcoords.shape)

    #cluster_centers_fcoords_neat = np.round(cluster_centers_fcoords_neat, 4)
    np.save(npyfile_name,cluster_centers_fcoords_neat)
    return cluster_centers_fcoords_neat
unit_cell =np.array([[ 100.0, 0.0, 0.0],
 [ 0.0, 100.0, 0.0],
 [ 0.0, 0.0, 100.0]])
#make 333 supercell of vvnode, for each line, looking for closest 6 atoms, distance range in cartesian coordinates shoud be around 12-15
#then find 6 center of the node in the 6 pairs
#remove the overlapping points by remove the lines by index
def clean_nodes_with_distance(cluster_centers_fcoords_neat,connected_dist_range=[11.1,15.5],cluster_size=6,savenpyfile='vvnode.npy'):
    #make 333 supercell of vvnode
    vvv333node = make_supercell_3x3x3(cluster_centers_fcoords_neat)
    vvv333node = np.vstack(vvv333node)
    # cluster analysis in cartesian coordinates
    array_vvvnode_ccords = np.dot(unit_cell, vvv333node.T).T
    matched_pairs = []
    centers = []
    overlap=[]
    check =[]
    #for each atom, find the closest 6 atoms, matched pair only once
    for i in range(len(cluster_centers_fcoords_neat)):
        idxs = None
        dist = None
        atom = array_vvvnode_ccords[i]
        dist = np.linalg.norm(array_vvvnode_ccords-atom,axis=1)
        idxs = np.argsort(dist)
        #check distance range of 
        if dist[idxs[cluster_size+1]] > connected_dist_range[1] and dist[idxs[cluster_size]] < connected_dist_range[1] and dist[idxs[1]]>connected_dist_range[0]:
            #find each pair center
            indices = idxs[1:cluster_size+1]
            for id in indices:
                if (i,id) in matched_pairs:
                    continue
                matched_pairs.append((id,i))
                matched_pairs.append((i,id))
                center = np.mean([array_vvvnode_ccords[i],array_vvvnode_ccords[id]],axis=0)
                centers.append(center)
        else:
            #remove overlapping points
            if dist[idxs[1]]<connected_dist_range[0]-1:
                if idxs[1] > i and idxs[1] < len(cluster_centers_fcoords_neat):
                    overlap.append(idxs[1])
                    print("error",i,dist[idxs[cluster_size+1]],dist[idxs[1]],dist[idxs[cluster_size]],idxs[:cluster_size+2])
            if dist[idxs[2]]<connected_dist_range[0]-1:
                if idxs[2] > i and idxs[2] < len(cluster_centers_fcoords_neat):
                    overlap.append(idxs[2])
                    print("error",i,dist[idxs[cluster_size+1]],dist[idxs[1]],dist[idxs[cluster_size]],idxs[:cluster_size+2])
            if dist[idxs[3]]<connected_dist_range[0]-1:
                if idxs[3] > i and idxs[3] < len(cluster_centers_fcoords_neat):
                    overlap.append(idxs[3])
                    print("error",i,dist[idxs[cluster_size+1]],dist[idxs[1]],dist[idxs[cluster_size]],idxs[:cluster_size+2])
    cluster_centers_fcoords_neat = cluster_centers_fcoords_neat[[r for r in range(len(cluster_centers_fcoords_neat)) if r not in overlap]]
    np.save(savenpyfile,np.vstack(cluster_centers_fcoords_neat))
    return cluster_centers_fcoords_neat


#final check
def final_check_nodes(cluster_centers_fcoords_neat,connected_dist_range=[11.1,15.5],cluster_size=6):
    vvv333node = make_supercell_3x3x3(cluster_centers_fcoords_neat)
    vvv333node = np.vstack(vvv333node)
        # cluster analysis in cartesian coordinates
    array_vvvnode_ccords = np.dot(unit_cell, vvv333node.T).T
    matched_pairs = []
    centers = []
    overlap=[]
    check =[]
    #for each atom, find the closest 6 atoms, matched pair only once
    for i in range(len(cluster_centers_fcoords_neat)):
        idxs = None
        dist = None
        atom = array_vvvnode_ccords[i]
        dist = np.linalg.norm(array_vvvnode_ccords-atom,axis=1)
        idxs = np.argsort(dist)
        #check distance range of 
        if dist[idxs[cluster_size+1]] > connected_dist_range[1] and dist[idxs[cluster_size]] <connected_dist_range[1] and dist[idxs[1]]>connected_dist_range[0]:
                #find each pair center
            indices = idxs[1:cluster_size+1]
            for id in indices:
                if (i,id) in matched_pairs:
                    continue
                matched_pairs.append((id,i))
                matched_pairs.append((i,id))
                center = np.mean([array_vvvnode_ccords[i],array_vvvnode_ccords[id]],axis=0)
                centers.append(center)
        else:
            print("error",i,dist[idxs[cluster_size+1]],dist[idxs[1]],dist[idxs[cluster_size]],idxs[:cluster_size+2])       
    return centers
#cluster_centers_fcoords_neat = analyze_metal_node(template_cif_file, "Cr", 3,npyfile_name="Cr_cluster_centers_fcoords_neat.npy")

In [ ]:
cluster_centers_fcoords_neat = analyze_metal_node(template_cif_file, "Cr", 3,npyfile_name="Cr_cluster_centers_fcoords_neat.npy")
cluster_centers_fcoords_neat = clean_nodes_with_distance(cluster_centers_fcoords_neat,connected_dist_range=[11.1,15.5],cluster_size=6,savenpyfile='vvnode.npy')

#use mod to unique the centers
centers = np.vstack(centers)
centers_fccords = np.dot(np.linalg.inv(unit_cell),centers.T).T
centers_fccords = np.mod(centers_fccords, 1)
centers_fccords = np.unique(centers_fccords,axis=0)
centers_fccords = np.round(centers_fccords,4)
np.save("eenode.npy",centers_fccords)
#for each atom, find the closest 5 atoms, these 6 atoms are the cluster, then find the center of the cluster, one atom can only be in one cluster

array = array_metal_ccords
centers = []
matched_atoms = []
for i in range(len(array)):
    if i in matched_atoms:
        continue
    atom = array[i]
    dist = np.linalg.norm(array-atom,axis=1)
    idx = np.argsort(dist)
    cluster = array[idx[:6]]
    matched_atoms.extend(idx[:6])
    center = np.mean(cluster,axis=0)
    centers.append(center)
    print(len(matched_atoms)/6)
#remove overlapping points
cluster_centers_ccoords = np.vstack(centers)
# cluster_centers should return fractional coordinates
cluster_centers_fcoords = np.dot(
    np.linalg.inv(unit_cell), cluster_centers_ccoords.T
).T
# filter cluster centers which is inside the unit cell, boundary condition is [-0.01,1.01]
# cluster_centers_fcoords = np.round(cluster_centers_fcoords,3)
cluster_centers_fcoords = np.mod(cluster_centers_fcoords, 1)
# cluster_centers_fcoords = [c for c in cluster_centers_fcoords if all([i>=-0.01 and i<=1.01 for i in c])]
cluster_centers_fcoords = filter_overlapping_points(cluster_centers_fcoords, 0.001)
cluster_centers_fcoords = np.round(cluster_centers_fcoords, 3)
# cluster_centers_fcoords = np.unique(cluster_centers_fcoords,axis=0)

np.save("eenode.npy",cluster_centers_fcoords)

In [ ]:
template_cif_file ='mil53_cleaved.cif'
#'I m m a'
import numpy as np
from src.MOF_builder._learn_template import extract_bridge_point_cluster_center_from_templatecif,extract_cluster_center_from_templatecif
vvnode,cell_info,unit_cell = extract_cluster_center_from_templatecif(template_cif_file, 'YY',1,2) # node com in template cif file, use fcluster to find cluster and the center of the cluster
eenode,_,_ = extract_cluster_center_from_templatecif(template_cif_file, 'XX',1,2) # edge com in template cif file, use fcluster to find the cluster and center of the cluster

#vvnode = np.unique(np.mod(vvnode,1),axis=0)
#eenode = np.unique(np.mod(eenode,1),axis=0)



In [26]:
template_cif_file ='irmof74.cif'
#'I m m a'
import numpy as np
from src.MOF_builder._learn_template import extract_bridge_point_cluster_center_from_templatecif,extract_cluster_center_from_templatecif
vvnode,cell_info,unit_cell = extract_cluster_center_from_templatecif(template_cif_file, 'Mg',1,2) # node com in template cif file, use fcluster to find cluster and the center of the cluster
eenode,_,_ = extract_cluster_center_from_templatecif(template_cif_file, 'YY',1,2) # edge com in template cif file, use fcluster to find the cluster and center of the cluster

vvnode = np.unique(np.mod(vvnode,1),axis=0)
eenode = np.unique(np.mod(eenode,1),axis=0)


P1 cell
P1 cell


In [27]:
vvnode,eenode

(array([[0.088, 0.666, 0.36 ],
        [0.28 , 0.361, 0.694],
        [0.422, 0.694, 0.334],
        [0.586, 0.306, 0.67 ],
        [0.728, 0.64 , 0.306],
        [0.92 , 0.333, 0.639]]),
 array([[0. , 0. , 0.5],
        [0. , 0.5, 0. ],
        [0.5, 0.5, 0. ],
        [0.5, 0.5, 0.5]]))

In [4]:
template_cif_file = 'irmof74841642.cif'
import numpy as np
from src.MOF_builder._learn_template import extract_bridge_point_cluster_center_from_templatecif,extract_cluster_center_from_templatecif
#vvnode,cell_info,unit_cell = extract_cluster_center_from_templatecif(template_cif_file, 'Mg',2,4) # node com in template cif file, use fcluster to find cluster and the center of the cluster
#eenode,_,_ = extract_cluster_center_from_templatecif(template_cif_file, 'E',2,3) # edge com in template cif file, use fcluster to find the cluster and center of the cluster


In [24]:
cluster_centers_fcoords = analyze_metal_node(template_cif_file, "Mg", 2,"mgnode.npy")
for i in range(15):
    if not final_check_nodes(cluster_centers_fcoords,connected_dist_range=[15.2,15.5],cluster_size=2):
        cluster_centers_fcoords = clean_nodes_with_distance(cluster_centers_fcoords,connected_dist_range=[15.3,15.5],cluster_size=2,savenpyfile='mgnode.npy')


apply 18  symmetry operation
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0

In [21]:
centers =final_check_nodes(cluster_centers_fcoords,connected_dist_range=[14,16],cluster_size=2)

error 0 34.739224228528755 33.42752907410298 33.45230447966179 [  0  17 198 256]
error 1 33.43273358551467 31.26420797013735 33.37508352049475 [  1 256  73 209]
error 2 34.554474022910526 5.071299143217647 33.432733585514654 [ 2 34 27 28]
error 3 34.73922422852876 33.42752907410298 33.45230447966178 [  3  19 385 463]
error 4 33.432733585514654 31.26420797013735 33.37508352049475 [  4 463  92 396]
error 5 33.43273358551467 5.0712991432176455 33.43273358551467 [ 5 54 56 53]
error 6 33.432733585514654 31.268669223361584 31.664620635655815 [ 6 19 54 44]
error 7 33.37971951649685 30.900269820828424 30.910899372227906 [ 7 10 13 19]
error 8 33.37508352049475 3.893337257418134 30.900269820828427 [ 8 79  9 67]
error 9 33.379719516496834 30.900269820828427 30.91089937222791 [ 9  8 12 17]
error 10 33.37508352049475 3.8933372574181444 30.900269820828424 [10 99  7 85]
error 11 31.664620635655808 0.0 31.26866922336159 [14 11 17 34]
error 12 31.66462063565581 5.071299143217647 30.91089937222791 [12 2

In [22]:
centers

[]

In [21]:
analyze_metal_node(template_cif_file, "E", 2,"enode.npy")

apply 18  symmetry operation
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0

array([[3.33350000e-01, 1.66650000e-01, 6.66650000e-01],
       [8.33350000e-01, 1.66650000e-01, 6.66650000e-01],
       [8.33350000e-01, 6.66650000e-01, 6.66650000e-01],
       [6.66650000e-01, 8.33350000e-01, 3.33350000e-01],
       [1.66650000e-01, 8.33350000e-01, 3.33350000e-01],
       [1.66650000e-01, 3.33350000e-01, 3.33350000e-01],
       [1.00000000e+00, 5.00000000e-01, 0.00000000e+00],
       [5.00000000e-01, 5.00000000e-01, 0.00000000e+00],
       [5.00000000e-01, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 5.00000000e-01, 1.21589216e-17],
       [5.00000000e-01, 5.00000000e-01, 1.21589216e-17],
       [5.00000000e-01, 1.00000000e+00, 1.21589216e-17],
       [3.33350000e-01, 1.66650000e-01, 6.66650000e-01],
       [8.33350000e-01, 1.66650000e-01, 6.66650000e-01],
       [8.33350000e-01, 6.66650000e-01, 6.66650000e-01],
       [6.66650000e-01, 8.33350000e-01, 3.33350000e-01],
       [1.66650000e-01, 8.33350000e-01, 3.33350000e-01],
       [1.66650000e-01, 3.33350

In [18]:
for i in range(len(vvnode)):
    for j in range(i+1,len(vvnode)):
        #check if close use np.isclose
        if np.isclose(vvnode[i],vvnode[j],atol=0.2).all():
            print(i,j,np.linalg.norm(vvnode[i]-vvnode[j]))
        

2 5 0.16830923919975402
2 7 0.1688164683909719
3 8 0.1679672587143102
4 6 0.16797023545854783
13 18 0.16796725871431017
14 19 0.1690118339052032
15 17 0.16797023545854775
16 19 0.168309239199754


In [ ]:
cell_info
Mgs = np.array([[0.727789    , 0.639617 ,    0.305894],
[0.088173 ,    0.666278  ,   0.360384],
[0.421895 ,    0.694106  ,   0.333722],
[0.280276 ,    0.360588  ,   0.693934],
[0.919688 ,    0.333345  ,   0.639412],
[0.586342 ,    0.306065  ,   0.670000]])
centers = []
for i in range(len(Mgs)):
    for j in range(len(Mgs)):
        dist = np.linalg.norm(np.dot(unit_cell,Mgs[i]-Mgs[j]))
        if dist <3.5 and i != j:
            center = np.mean([Mgs[i],Mgs[j]],axis=0)
            print(i,j,dist,center)
            centers.append(center)

In [ ]:
from src.MOF_builder._learn_template import make_supercell_3x3x3
centers = []
node333 = make_supercell_3x3x3(vvnode)
for i in range(len(node333)):
    for j in range(len(node333)):
        dist = np.linalg.norm(np.dot(unit_cell,(node333[i]-node333[j])))
        if dist <4.5 and i != j:
            print(i,j,dist)
            center = np.mean([node333[i],node333[j]],axis=0)
            centers.append(center)
centers = np.vstack(centers)
centers = np.mod(centers,1)
centers = np.round(centers,5)
centers = np.unique(centers,axis=0)


In [ ]:
eenode =np.unique(np.mod(eenode,1),axis=0)

In [ ]:
eenode

In [ ]:
#read vvnode.npy to Vnodes 

import numpy as np
import os
import re
import spglib 

# make a dict of space group info
# key: international_short
# value: space group info
# space group info: (number, international_short, international_full, international, schoenflies, hall)
space_group_info_dict = {}
for i in range(1, 531):
    space_group_info_dict[spglib.get_spacegroup_type(i)["international_short"]] = spglib.get_spacegroup_type(i)

with open('/Users/chenxili/GitHub/vlx_mofbuilder/data/RCSRnets-2019-06-01.cgd') as f:
    lines = f.readlines()
#extract the space group info from the cgd file, start with CRYSTAL end with END
starts = []
ends = []
for i, line in enumerate(lines):
    if "CRYSTAL" in line or "crystal" in line:
        if "#" in line:
            continue
        starts.append(i)
    if "END" in line or "end" in line:
        if "#" in line:
            continue
        end = i
        ends.append(i)
#check if the number of starts and ends are the same
if len(starts) != len(ends):
    print("Error: the number of starts and ends are not the same")

#extract the space group info from the cgd file
crystal_infos = []
for i in range(len(starts)):
    crystal_info = []
    if ends[i] - starts[i] < 1:
        print("Error: the crystal info is not complete",starts[i],ends[i],lines[starts[i]:ends[i]+1])
        continue

    for j in range(starts[i], ends[i]+1):
        crystal_info.append(lines[j])
    crystal_infos.append(crystal_info)
def similar_group_name(group):
    #make first letter uppercase
    if ":" in group:
        group = group.split(":")[0]
    return re.sub(r'[^a-zA-Z0-9]', '', group)

#https://cci.lbl.gov/sginfo/hall_symbols.html
#2d https://pscf.readthedocs.io/en/latest/groups.html
def fetch_group_number(group):
    numbers = []
    if group =="C12/m1":
        numbers = [71]
        return numbers
    if group =="Cmca":
        numbers = [304] #? a/e
        return numbers
    if group =="C2mm":
        numbers = [188]
        return numbers
    if group =="P121/n1":
        numbers = [89]
        return numbers

    if group =="Ccca:2":
        numbers = [325] #? a/e
        return numbers
    if group =="Cmma":
        numbers = [317] #? a/e
        return numbers
    if group =="C12/c1":
        numbers = [107]
        return numbers
    if group =="I12/a1":
        numbers = [107]
        return numbers
    if group =="I12/m1":
        numbers = [71]
        return numbers
    if group =="P121/c1":
        numbers = [89]
        return numbers
    #if group =="llw-z":
    #    numbers = [196]
    #    return numbers
    if group =="R-3:H":
        numbers = [148]
        return numbers
    if group =="R3m:H":
        numbers = [160]
        return numbers

    if group =="R3c:H":
        numbers = [161]
        return numbers

    if group =="R-3m:H":
        numbers = [166]
        return numbers
    if group =="R-3c:H":
        numbers = [167]
        return numbers

    if group[0] =='p':
        #print("2d group")
        return [-1]
    if group[0] =='c':
        #print("2d group")
        return [-1]

    for key in space_group_info_dict.keys():
        if group == key:
            numbers=[space_group_info_dict[key].hall_number]
            return numbers
        elif similar_group_name(group) == similar_group_name(key):
            #print('hall number:', space_group_info_dict[key])
            numbers.append(space_group_info_dict[key].hall_number)
    if len(numbers) >1:
        print()
        print('more than one space group found',group,numbers)
        return [-1]
    elif len(numbers) == 0:
        print('*'*10)
        print('no space group found',group)
        print('*'*10)
        return [-1]
    return numbers




In [ ]:
fetch_group_number("R3")

In [28]:
import datetime
def write_cif_nobond(lines, params, cifname,infoline):
    a,b,c,alpha,beta,gamma = params
    os.makedirs('output_cifs',exist_ok=True)
    opath = os.path.join('output_cifs', cifname)
    print(opath)
	
    with open(opath, 'w') as new_cif:
         new_cif.write('data_' + cifname[0:-4] +infoline+ '\n')
         new_cif.write('_audit_creation_date              ' + datetime.datetime.today().strftime('%Y-%m-%d') + '\n')
         new_cif.write("_audit_creation_method            'MOFbuilder_1.0'" + '\n')
         new_cif.write("_symmetry_space_group_name_H-M    'P1'" + '\n')
         new_cif.write('_symmetry_Int_Tables_number       1' + '\n')
         new_cif.write('_symmetry_cell_setting            triclinic' + '\n')
         new_cif.write('loop_' + '\n')
         new_cif.write('_symmetry_equiv_pos_as_xyz' + '\n')
         new_cif.write('  x,y,z' + '\n')
         if float(a) < 10:
             a = float(a)*1
             b = float(b)*1
             c = float(c)*1
         new_cif.write('_cell_length_a                    ' + str(a) + '\n')
         new_cif.write('_cell_length_b                    ' + str(b) + '\n')
         new_cif.write('_cell_length_c                    ' + str(c) + '\n')
         new_cif.write('_cell_angle_alpha                 ' + str(alpha) + '\n')
         new_cif.write('_cell_angle_beta                  ' + str(beta) + '\n')
         new_cif.write('_cell_angle_gamma                 ' + str(gamma) + '\n')
         new_cif.write('loop_' + '\n')
         new_cif.write('_atom_site_label' + '\n')
         new_cif.write('_atom_site_type_symbol' + '\n')
         new_cif.write('_atom_site_fract_x' + '\n')
         new_cif.write('_atom_site_fract_y' + '\n')
         new_cif.write('_atom_site_fract_z' + '\n')
         
         
         new_cif.writelines(lines)
         
         new_cif.write('loop_' + '\n')


In [ ]:
#IRMOF-74 BOND LENGTH 12.11-13.2 1 CONNECTED, 2O shared

In [29]:
import os
#fetch_group_number("Fd-3m")
#"Imma" #mil53
#"Fd-3m" #mil101`
Vnodes = vvnode#np.load('vvnode.npy')
edge_centers = eenode#np.load('eenode.npy')
node_positions = np.array(Vnodes).reshape((-1,3)).astype(float)
edge_positions = np.array(edge_centers).astype(float)
all_lines = []
# Print nodes
for i, pos in enumerate(node_positions, start=1):
    #print(f"V{i:<3}  V   {pos[0]:.5f}  {pos[1]:.5f}  {pos[2]:.5f}")
    all_lines.append(f"V{i:<3}  V   {pos[0]:.5f}  {pos[1]:.5f}  {pos[2]:.5f}\n")

# Print edges
for i, pos in enumerate(edge_positions, start=1):
    #print(f"E{i:<3}  E   {pos[0]:.5f}  {pos[1]:.5f}  {pos[2]:.5f}")
    all_lines.append(f"E{i:<3}  E   {pos[0]:.5f}  {pos[1]:.5f}  {pos[2]:.5f}\n")
infoline = f'_{"R3"}  hall_number: {434}, V_con: {2}'
write_cif_nobond(all_lines, cell_info,'00irmof74.cif',infoline)

output_cifs/00irmof74.cif


In [ ]:
vvnode_double = vvnode.copy()
vvnode_double[:,0] = vvnode_double[:,0]/2
vvnode_shift = vvnode_double.copy()
vvnode_shift[:,0] = vvnode_shift[:,0]+0.5
vvnode_com = np.concatenate((vvnode_double,vvnode_shift),axis=0)
for i in range(len(vvnode_com)):
    print(f"V{i:<3}  V   {vvnode_com[i][0]:.5f}  {vvnode_com[i][1]:.5f}  {vvnode_com[i][2]:.5f}")

In [ ]:
for i in range(len(vvnode)):
    print(f"V{i:<3}  V   {vvnode[i][0]:.5f}  {vvnode[i][1]:.5f}  {vvnode[i][2]:.5f}")